In [1]:
import pandas as pd 
import numpy as np
import sqlite3
conn = sqlite3.connect('CourseData2.db')
c = conn.cursor()

In [2]:
# Creation of DataBase

In [3]:
%load_ext sql

In [4]:
%%sql
sqlite:///CourseData2.db

'Connected: @CourseData2.db'

In [5]:
%%sql
--Dropping tables for when we need to restart and clear CourseData database-- 
--Drop ERD tables--
DROP TABLE IF EXISTS SECTION;
DROP TABLE IF EXISTS CATALOG;
DROP TABLE IF EXISTS INSTRUCTOR;
DROP TABLE IF EXISTS COURSE_MEETING;
DROP TABLE IF EXISTS PROGRAM;

--Drop import tables--
DROP TABLE IF EXISTS import_course;
DROP TABLE IF EXISTS import_course_meeting;
DROP TABLE IF EXISTS import_course_catalog;

 * sqlite:///CourseData2.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

## Create Tables (Import and ERD) ##

In [6]:
%%sql
--Creating import_catalog table-- 

CREATE TABLE 'import_course_catalog' (
    program_code TEXT NOT NULL,
    program_name VARCHAR NOT NULL,
    catalog_id VARCHAR NOT NULL,
    course_title VARCHAR NOT NULL, 
    credits VARCHAR NOT NULL,
    prereqs VARCHAR, 
    coreqs VARCHAR,
    fees VARCHAR,
    attributes VARCHAR,
    description VARCHAR 
);

 * sqlite:///CourseData2.db
Done.


[]

In [7]:
%%sql
--Creating import_course_meeting table-- 

CREATE TABLE 'import_course_meeting' (
    term VARCHAR NOT NULL,
    crn INTEGER NOT NULL,
    location VARCHAR NOT NULL,
    day TEXT NOT NULL,
    start VARCHAR NOT NULL,
    end VARCHAR NOT NULL 
);

 * sqlite:///CourseData2.db
Done.


[]

In [8]:
%%sql
--Creating import_course table-- 

CREATE TABLE 'import_course' (
    term VARCHAR NOT NULL,
    crn INTEGER NOT NULL,
    catalog_id VARCHAR NOT NULL,
    section VARCHAR NOT NULL,
    credits VARCHAR,
    title VARCHAR, 
    meetings VARCHAR,
    timecodes VARCHAR, 
    primary_instructor TEXT,
    cap INTEGER, 
    act INTEGER, 
    rem INTEGER 
);

 * sqlite:///CourseData2.db
Done.


[]

In [9]:
%%sql
-- Creating table INSTRUCTOR with surrogate primary key-- 

CREATE TABLE 'INSTRUCTOR' (
    InID INTEGER NOT NULL PRIMARY KEY,
    Name TEXT NOT NULL
);


--Creating table PROGRAM with PID as the primary key-- 
CREATE TABLE 'PROGRAM'(
    PID INTEGER PRIMARY KEY,
    PName TEXT NOT NULL,
    PCode TEXT NOT NULL
);

--Creating table CATALOG with CatID as the primary key-- 

CREATE TABLE 'CATALOG'(
    CatClass INTEGER PRIMARY KEY,
    CatID VARCHAR,
    Title VARCHAR NOT NULL,
    Description TEXT,
    Credits TEXT NOT NULL,
    PID INTEGER NOT NULL,
    PREREQS TEXT,
    COREQS TEXT,
    FEES INTEGER,
    Attribute TEXT,
    FOREIGN KEY (PID) REFERENCES PROGRAM(PID)
    
);

--Creating table SECTION with surrogate primary key-- 

CREATE TABLE 'SECTION'(
    SID INTEGER NOT NULL PRIMARY KEY,
    CRN INTEGER NOT NULL,
    Term TEXT NOT NULL,
    Section VARCHAR NOT NULL,
    Cap INTEGER,
    Act INTEGER,
    Rem INTEGER,
    CatID VARCHAR,
    InID INTEGER,
    FOREIGN KEY (InID) REFERENCES INSTRUCTOR(InID),
    FOREIGN KEY (CatID) REFERENCES Catalog(CatID)
);


--Creating table COURSE_MEETING with MID as the Primary Key-- 

CREATE TABLE 'COURSE_MEETING' (
    MID INTEGER PRIMARY KEY,
    CRN INTEGER NOT NULL,
    Term TEXT NOT NULL,
    Location VARCHAR NOT NULL,
    Start TEXT, 
    End TEXT, 
    DAY TEXT,
    FOREIGN KEY (CRN) REFERENCES SECTION(CRN),
    FOREIGN KEY (TERM) REFERENCES SECTION(TERM)
);

 * sqlite:///CourseData2.db
Done.
Done.
Done.
Done.
Done.


[]

## Populate the Import Tables with the Data from the csv files ##

**Creating dataframes for the import course**

In [10]:
## populating import_course table
fall = ['SourceData/Fall2014/courses.csv','SourceData/Fall2015/courses.csv', 'SourceData/Fall2016/courses.csv',
        'SourceData/Fall2017/courses.csv','SourceData/Fall2018/courses.csv']
dff0 = pd.read_csv(fall[0])
dff1 = pd.read_csv(fall[1])
dff2 = pd.read_csv(fall[2])    
dff3 = pd.read_csv(fall[3])
dff4 = pd.read_csv(fall[4])

In [11]:
df_fall = pd.concat([dff0, dff1, dff2, dff3, dff4], axis=0)
df_fall.sample(5)

,term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem
1339,Fall2017,72938,RES 0310,01,3.000,"Capstone Seminar: Current Topics in Russia, Ea...",[],[],David W. McFadden,0,2,-2
155,Fall2017,71535,BI 0170P,09,0.000,General Biology I PLG,"[{'days': 'R', 'times': '0300pm-0350pm', 'date...",['R 0300pm-0350pm 09/05-12/11 BNW 319'],TBA,10,9,1
897,Fall2018,75910,FT 0559,03,3.000,Practicum in Family Therapy I,"[{'days': 'T', 'times': '0200pm-0700pm', 'date...",['T 0200pm-0700pm 09/04-12/17 SWH 102'],Patricia Macomber,8,6,2
190,Fall2016,78733,BB 0115,01,0.000,SHRM Learning Systems: Members,"[{'days': 'T', 'times': '0630pm-0930pm', 'date...",['T 0630pm-0930pm 09/13-11/29 CNS 2'],Raymond T. Lamanna,20,9,11
118,Fall2015,75279,BI 0151,A,4.000,Elements of Microbiology,"[{'days': 'MR', 'times': '1230pm-0145pm', 'dat...",['MR 1230pm-0145pm 09/01-12/07 BNW 131'],Eugenia T. Zavras,48,47,1


In [12]:
df_fall.describe()

,crn,cap,act,rem
count,7486.000000,7486.00000,7486.000000,7486.000000
mean,74523.769169,18.39487,16.310446,2.219076
std,2512.545835,14.41373,9.863494,17.175757
min,70001.000000,0.00000,-1.000000,-75.000000
25%,72213.000000,12.00000,8.000000,0.000000
50%,74848.500000,19.00000,16.000000,1.000000
75%,76698.750000,25.00000,24.000000,5.000000
max,79215.000000,500.00000,90.000000,999.000000


In [13]:
spring = ['SourceData/Spring2015/courses.csv','SourceData/Spring2016/courses.csv', 'SourceData/Spring2017/courses.csv',
        'SourceData/Spring2018/courses.csv','SourceData/Spring2019/courses.csv', 'SourceData/SpringBreak2017/courses.csv']
dfs0 = pd.read_csv(spring[0])
dfs1 = pd.read_csv(spring[1])
dfs2 = pd.read_csv(spring[2])    
dfs3 = pd.read_csv(spring[3])
dfs4 = pd.read_csv(spring[4])
dfs5 = pd.read_csv(spring[5])

In [14]:
df_spring = pd.concat([dfs0,dfs1,dfs2,dfs3,dfs4,dfs5], axis=0)
df_spring.sample(5)

,term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem
1363,Spring2016,38110,SL 0441,01,3.000,Teaching and Learning within Multicultural Con...,"[{'days': 'R', 'times': '0715pm-0915pm', 'date...",['R 0715pm-0915pm 01/21-05/12 CNS 9'],Stephanie L. Storms,10,11,-1
73,Spring2015,31978,AE 0291,B,3.000,Business Ethics,"[{'days': 'T', 'times': '0630pm-0900pm', 'date...",['T 0630pm-0900pm 01/20-04/30 DMH 350'],Scott W. Ventrella,29,30,-1
649,Spring2016,38010,HI 0285,A,3.000,Modern China:1800 to Present,"[{'days': 'TF', 'times': '1100am-1215pm', 'dat...",['TF 1100am-1215pm 01/19-05/03 SON 203'],Danke Li,25,22,3
365,Spring2016,37848,ED 0581,05,6.000,Directed Observation and Supervised Student Te...,[],[],Bridget M. Fine,1,1,0
1253,Spring2019,38824,PY 0351,01,3.000,Senior Seminar: False Memories,"[{'days': 'MR', 'times': '0200pm-0315pm', 'dat...",['MR 0200pm-0315pm 01/22-04/30 BNW 334'],Linda A. Henkel,15,14,1


In [15]:
df_spring.describe()

,crn,cap,act,rem
count,7122.000000,7122.000000,7122.000000,7122.000000
mean,35212.001966,18.123982,15.658804,2.465178
std,2618.670948,10.876640,10.342885,6.497100
min,31001.000000,0.000000,0.000000,-47.000000
25%,32773.250000,11.000000,7.000000,0.000000
50%,34835.000000,19.000000,16.000000,1.000000
75%,37693.750000,25.000000,24.000000,4.000000
max,41702.000000,100.000000,90.000000,89.000000


In [16]:
summer = ['SourceData/Summer2015/courses.csv', 'SourceData/Summer2016/courses.csv',
        'SourceData/Summer2017/courses.csv','SourceData/Summer2018/courses.csv']
dfsm0 = pd.read_csv(summer[0])
dfsm1 = pd.read_csv(summer[1])
dfsm2 = pd.read_csv(summer[2])    
dfsm3 = pd.read_csv(summer[3])

In [17]:
df_summer = pd.concat([dfsm0,dfsm1,dfsm2,dfsm3], axis=0)
df_summer.sample(5)

,term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem
61,Summer2017,57402,CN 0446,01,3.000,Spirituality and Counseling,"[{'days': 'TR', 'times': '0400pm-0830pm', 'dat...",['TR 0400pm-0830pm 06/07-07/07 CNS 9'],Tracey Robert,20,17,3
149,Summer2015,55326,MG 0503,01,3.000,Legal and Ethical Environment of Business,"[{'days': 'MTWR', 'times': '0600pm-0930pm', 'd...","['MTWR 0600pm-0930pm 05/18-05/21 DSB 115', 'S ...",David P. Schmidt,25,12,13
144,Summer2018,58205,MK 0580,OL1,3.000,Multivariate Data Analysis for Decision Making,[],[],Michael R. Sciandra,20,6,14
73,Summer2018,58006,EC 0011,OL1,3.000,Introduction to Microeconomics,[],[],William F. Vasquez Mazariegos,20,8,12
170,Summer2016,56266,ME 0452,01,3.000,Heat and Mass Transfer,"[{'days': 'T', 'times': '1230pm-0430pm', 'date...",['T 1230pm-0430pm 05/24-07/26 BNW 254'],Shahrokh Etemad,25,13,12


In [18]:
df_summer.describe()

,crn,cap,act,rem
count,1224.000000,1224.000000,1224.000000,1224.000000
mean,56704.305556,14.843954,10.913399,3.928922
std,1143.898206,10.771969,9.124774,6.881239
min,55001.000000,0.000000,-1.000000,-24.000000
25%,55382.750000,6.000000,5.000000,0.000000
50%,56517.500000,16.000000,9.000000,3.000000
75%,58008.500000,20.000000,15.000000,9.000000
max,58477.000000,60.000000,59.000000,24.000000


In [19]:
winter = ['SourceData/Winter2015/courses.csv', 'SourceData/Winter2016/courses.csv',
        'SourceData/Winter2017/courses.csv','SourceData/Winter2018/courses.csv']
dfw0 = pd.read_csv(winter[0])
dfw1 = pd.read_csv(winter[1])
dfw2 = pd.read_csv(winter[2])    
dfw3 = pd.read_csv(winter[3])

In [20]:
df_winter = pd.concat([dfw0,dfw1,dfw2,dfw3], axis=0)
df_winter.sample(5)

,term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem
6,Winter2016,11622,CO 0246,A,3.000,Family Communication,"[{'days': 'MTWRF', 'times': '0830am-0430pm', '...",['MTWRF 0830am-0430pm 01/06-01/12 DMH 350'],Margaret A. Wills,15,15,0
4,Winter2015,10911,BU 0211,A,3.000,Legal Environment of Business,"[{'days': 'MTWRF', 'times': '0830am-0430pm', '...",['MTWRF 0830am-0430pm 01/05-01/09 DSB 109'],Sharlene A. McEvoy,15,16,-1
2,Winter2015,10893,BI 0075,01,3.000,Ecology and Society,[],[],Thomas B. Cunningham,20,22,-2
4,Winter2017,11707,CO 0100,OL1,3.000,Human Communication Theories,[],[],David Gudelunas,20,19,1
11,Winter2016,11632,EE 0352,JD,3.000,Digital Communications,[],[],Jeffrey N. Denenberg,0,0,0


In [21]:
df_winter.describe()

,crn,cap,act,rem
count,105.000000,105.000000,105.000000,105.000000
mean,11524.266667,17.133333,12.342857,4.790476
std,355.382575,7.612069,6.584990,6.207463
min,10893.000000,0.000000,0.000000,-3.000000
25%,11603.000000,15.000000,9.000000,0.000000
50%,11633.000000,20.000000,12.000000,4.000000
75%,11802.000000,20.000000,18.000000,9.000000
max,11841.000000,30.000000,25.000000,27.000000


In [22]:
df_course = pd.concat([df_fall, df_spring, df_summer, df_winter], axis=0)
df_course.sample(5)

,term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem
989,Spring2015,32841,NS 0323,S,4.000,Pediatric Nursing,"[{'days': 'F', 'times': '0945am-1245pm', 'date...",['F 0945am-1245pm 01/15-04/24 BNW 254'],Eileen R. O'Shea,42,42,0
395,Spring2019,37642,ED 0431,01,3.000,Extending Literacy in the Elementary School: G...,"[{'days': 'M', 'times': '0800am-1030am', 'date...",['M 0800am-1030am 01/22-05/10 CNS 108'],Ryan P. Colwell,20,10,10
288,Spring2016,37557,CN 0099,01,0.000,Comp Exam: Clin/Mntl Hlth Coun,"[{'days': 'W', 'times': '0830am-0100pm', 'date...",['W 0830am-0100pm 04/13-04/13 CNS 202'],JoAnn M. O'Connell,20,13,7
892,Fall2018,74700,FT 0553,01,3.000,Family Therapy Pre-Practicum,"[{'days': 'T', 'times': '1130am-0130pm', 'date...",['T 1130am-0130pm 09/04-12/17 SWH 102'],Kerri A. Anderson,8,4,4
1305,Fall2016,77805,MPA 0410,01,3.000,Introduction to Public Administration,"[{'days': 'T', 'times': '0630pm-0930pm', 'date...",['T 0630pm-0930pm 09/06-12/12 DMH 231'],Gayle A. Alberda,15,12,3


In [23]:
df_course.describe()

,crn,cap,act,rem
count,15937.000000,15937.000000,15937.000000,15937.000000
mean,55172.299178,17.992784,15.578591,2.477317
std,19308.879968,12.671906,10.107690,12.709781
min,10893.000000,0.000000,-1.000000,-75.000000
25%,35426.000000,10.000000,7.000000,0.000000
50%,57145.000000,19.000000,15.000000,1.000000
75%,74541.000000,25.000000,23.000000,5.000000
max,79215.000000,500.000000,90.000000,999.000000


In [24]:
df_course.to_sql('import_course', conn, if_exists='append', index=False)

In [25]:
%%sql
SELECT *
FROM import_course
LIMIT 3;

 * sqlite:///CourseData2.db
Done.


term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem
Fall2014,70384,AC 0011,C01,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '0800am-0915am', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 0800am-0915am 09/02-12/08 DSB 105'],Michael P. Coyne,0,31,-31
Fall2014,70385,AC 0011,C02,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '0930am-1045am', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 0930am-1045am 09/02-12/08 DSB 105'],Michael P. Coyne,0,31,-31
Fall2014,70382,AC 0011,C03,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '1230pm-0145pm', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 1230pm-0145pm 09/02-12/08 DSB 105'],Michael P. Coyne,0,31,-31


**Creating dataframes for the import course_meeting**

In [26]:
cm_fall = ['SourceData/Fall2014/course_meetings.csv', 'SourceData/Fall2015/course_meetings.csv',
           'SourceData/Fall2016/course_meetings.csv', 'SourceData/Fall2017/course_meetings.csv',
          'SourceData/Fall2018/course_meetings.csv']
dfcmf1 = pd.read_csv(cm_fall[0])
dfcmf2  = pd.read_csv(cm_fall[1])
dfcmf3  = pd.read_csv(cm_fall[2])
dfcmf4  = pd.read_csv(cm_fall[3])
dfcmf5  = pd.read_csv(cm_fall[4])

In [27]:
df_cmfall = pd.concat([dfcmf1, dfcmf2, dfcmf3, dfcmf4, dfcmf5], axis=0)
df_cmfall.sample(5)

,term,crn,location,day,start,end
6674,Fall2015,75007,CNS 15,R,2015-10-29T09:00:00,2015-10-29T09:50:00
2344,Fall2016,77584,BNW 245,M,2016-09-26T14:00:00,2016-09-26T16:50:00
14614,Fall2016,77162,DSB 106,R,2016-10-06T15:30:00,2016-10-06T16:45:00
1974,Fall2015,76865,BLM LL105,M,2015-10-26T14:00:00,2015-10-26T15:15:00
8369,Fall2017,71018,CNS 2,F,2017-11-03T09:30:00,2017-11-03T10:45:00


In [28]:
cm_spring = ['SourceData/Spring2015/course_meetings.csv',
           'SourceData/Spring2016/course_meetings.csv', 'SourceData/Spring2017/course_meetings.csv',
          'SourceData/Spring2018/course_meetings.csv', 'SourceData/Spring2019/course_meetings.csv']
dfcms1 = pd.read_csv(cm_spring[0])
dfcms2  = pd.read_csv(cm_spring[1])
dfcms3  = pd.read_csv(cm_spring[2])
dfcms4  = pd.read_csv(cm_spring[3])
dfcms5  = pd.read_csv(cm_spring[4])


In [29]:
df_cmspring = pd.concat([dfcms1, dfcms2, dfcms3, dfcms4, dfcms5], axis=0)
df_cmspring.sample(5)

,term,crn,location,day,start,end
5674,Spring2019,37708,NHS 411,R,2019-04-04T11:00:00,2019-04-04T12:15:00
2895,Spring2016,37467,BNW 261,R,2016-04-14T14:00:00,2016-04-14T16:50:00
9097,Spring2019,38941,CNS 103,W,2019-03-27T11:00:00,2019-03-27T13:30:00
26028,Spring2016,38182,CNS 103,F,2016-03-25T14:00:00,2016-03-25T15:15:00
25865,Spring2018,35213,BNW 253,W,2018-03-21T17:00:00,2018-03-21T18:15:00


In [30]:
cm_springbreak = ['SourceData/SpringBreak2017/course_meetings.csv']

dfcmsb1 = pd.read_csv(cm_springbreak[0])


In [31]:
cm_summer = ['SourceData/Summer2015/course_meetings.csv',
           'SourceData/Summer2016/course_meetings.csv', 'SourceData/Summer2017/course_meetings.csv',
          'SourceData/Summer2018/course_meetings.csv']
dfcmsu1 = pd.read_csv(cm_summer[0])
dfcmsu2  = pd.read_csv(cm_summer[1])
dfcmsu3  = pd.read_csv(cm_summer[2])
dfcmsu4  = pd.read_csv(cm_summer[3])

In [32]:
df_cmsummer = pd.concat([dfcmsu1, dfcmsu2, dfcmsu3, dfcmsu4], axis=0)
df_cmspring.sample(5)

,term,crn,location,day,start,end
19040,Spring2015,32458,YAL,W,2015-03-04T12:30:00,2015-03-04T19:30:00
16670,Spring2016,37162,DSB 114,T,2016-02-23T09:30:00,2016-02-23T10:45:00
331,Spring2016,37815,DSB 110B,R,2016-04-21T17:00:00,2016-04-21T18:15:00
25974,Spring2019,37728,CNS 10,R,2019-02-14T08:00:00,2019-02-14T08:50:00
60,Spring2017,31092,DSB 108,R,2017-02-09T11:00:00,2017-02-09T12:15:00


In [33]:
cm_winter = ['SourceData/Winter2015/course_meetings.csv',
           'SourceData/Winter2016/course_meetings.csv', 'SourceData/Winter2017/course_meetings.csv',
          'SourceData/Winter2018/course_meetings.csv']
dfcmw1 = pd.read_csv(cm_winter[0])
dfcmw2  = pd.read_csv(cm_winter[1])
dfcmw3  = pd.read_csv(cm_winter[2])
dfcmw4  = pd.read_csv(cm_winter[3])

In [34]:
df_cmwinter = pd.concat([dfcmw1, dfcmw2, dfcmw3, dfcmw4], axis=0)
df_cmwinter.sample(5)

,term,crn,location,day,start,end
39,Winter2018,11834,DSB 106,W,2018-01-10T09:00:00,2018-01-10T17:30:00
179,Winter2017,11726,CNS 108,R,2017-01-05T09:00:00,2017-01-05T15:00:00
52,Winter2015,10909,DSB 111,W,2015-01-07T18:00:00,2015-01-07T21:30:00
27,Winter2016,11601,DSB 107,R,2016-01-07T08:30:00,2016-01-07T16:30:00
74,Winter2017,11720,CNS 202,W,2017-01-04T09:00:00,2017-01-04T15:00:00


In [35]:
df_coursemeeting = pd.concat([df_cmfall,df_cmspring,dfcmsb1,df_cmsummer,df_cmwinter],axis=0)

In [36]:
df_coursemeeting.to_sql('import_course_meeting', conn, if_exists='append', index=False)

In [37]:
%%sql
SELECT *
FROM import_course_meeting
LIMIT 5;

 * sqlite:///CourseData2.db
Done.


term,crn,location,day,start,end
Fall2014,70384,DSB 105,T,2014-09-02T08:00:00,2014-09-02T09:15:00
Fall2014,70384,DSB 105,F,2014-09-05T08:00:00,2014-09-05T09:15:00
Fall2014,70384,DSB 105,T,2014-09-09T08:00:00,2014-09-09T09:15:00
Fall2014,70384,DSB 105,F,2014-09-12T08:00:00,2014-09-12T09:15:00
Fall2014,70384,DSB 105,T,2014-09-16T08:00:00,2014-09-16T09:15:00


**Creating the dataframes for import_course_catalog**

In [38]:
cc2017_2018 = ['SourceData/Catalogs/CourseCatalog2017_2018.csv']
dfcc17_18 = pd.read_csv(cc2017_2018[0])
dfcc17_18.sample(5)

,program_code,program_name,catalog_id,course_title,credits,prereqs,coreqs,fees,attributes,description
1408,HI,History,HI 0240,The Personal Is Political: Women's Activism in...,3 Credits,HI 0010 or CL 0115 or CL 0116.,NaN,NaN,"ASHI American Studies: History, ASUP American ...",Little fanfare and much derision accompanied t...
841,LA,Latin,LA 0323,Latin Prose I,3 Credits,LA 0211.,NaN,NaN,NaN,Students undertake extensive readings of selec...
2152,EN,English,EN 0291,Gender and Sexualilty in Film and Literature,3 Credits,One 100-level English literature course.,NaN,NaN,"E_AF English Literature After 1800, WSGF Women...",This course examines the way gender and sexual...
977,MU,Music,MU 0087,Private Lessons: Trombone,2 Credits,NaN,NaN,$575 Music Lesson Fee,NaN,NaN
1522,HI,History,HI 0251,The American Century: The United States and th...,3 Credits,HI 0010 or CL 0115 or CL 0116.,NaN,NaN,"ASGW American Studies: Gateway, ASHI American ...","This course examines the development, crises, ..."


In [39]:
cc2018_2019 = ['SourceData/Catalogs/CourseCatalog2018_2019.csv']
dfcc18_19 = pd.read_csv(cc2018_2019[0])
dfcc18_19.sample(5)

,program_code,program_name,catalog_id,course_title,credits,prereqs,coreqs,fees,attributes,description
625,PO,Politics,PO 0134,Globalization: Who Rules the World?,3 Credits,NaN,NaN,NaN,PMIR Politics Major: International Relations,"In a globalizing world, understanding the link..."
1513,FTM,"Film, Television, and Media Arts",FTM 0205,Survey of Film Music: Hearing the Movies,3 Credits,FTM 0010 or MU 0103 or MU 0104.,NaN,NaN,VPCH Visual and Performing Arts Core: History ...,This course provides an overview of film music...
1716,MD,Educational Technology,MD 0470,Designing for Online Instruction,3 Credits,MD 0400.,NaN,NaN,NaN,This course examines the nature of teaching on...
572,SP,Spanish,SP 0341,20th Century Spanish Literature,3 Credits,"SP 0245, SP 0251.",NaN,NaN,NaN,This course examines works and literary moveme...
66,AS,American Studies,AS 0486,"Health and Healing in America, 1650 to 2000",3 Credits,NaN,NaN,NaN,NaN,This seminar explores the idea of America thro...


In [40]:
df_catalog = pd.concat([dfcc17_18,dfcc18_19], axis=0)
df_catalog.sample(5)

,program_code,program_name,catalog_id,course_title,credits,prereqs,coreqs,fees,attributes,description
1311,HI,History,HI 0230,Early Modern France,3 Credits,HI 0010 or CL 0115 or CL 0116.,NaN,NaN,"FREN French Course Taught in English, H_BF His...","This course covers the political, social, and ..."
532,PS,Physics,PS 0076,Physics of Sound and Music,3 Credits,NaN,NaN,NaN,EDCG Educational Studies Cognate,"Designed for the non-science major, this cours..."
1045,NS,Nursing,NS 0307L,Fundamentals of Nursing Care Lab,0 Credits,NaN,NaN,NaN,NaN,NaN
1593,ECE,Electrical and Computer Engineering,ECE 0477,Power Security and Reliability,3 Credits,ECE 0495.,NaN,NaN,NaN,This course focuses on Power System Protection...
991,IS,Information Systems,IS 0315,Data Mining and Applications,3 Credits,IS 0210.,NaN,NaN,BUEL Business Elective,This course provides the students with an unde...


In [41]:
df_catalog.to_sql('import_course_catalog', conn, if_exists='append', index=False)

In [42]:
%%sql
SELECT *
FROM import_course_catalog
LIMIT 10;

 * sqlite:///CourseData2.db
Done.


program_code,program_name,catalog_id,course_title,credits,prereqs,coreqs,fees,attributes,description
AN,Asian Studies,AN 0301,Independent Study,1-3 Credits,None,None,None,None,Students undertake an individualized program of study in consultation with a director from the Asian studies faculty.
AN,Asian Studies,AN 0310,Asian Studies Seminar,3 Credits,None,None,None,None,"This seminar examines selected topics concerning Asia. This course is taught in conjunction with another 100-300 level course from a rotation of course offerings. Consult the Asian Studies director to identify the conjoined course for a given semester. The seminar concentrates on topics within the parameters of the conjoined course syllabus but adds research emphasis. Students registered for this course must complete a research project, to include 300-level research, in addition to the regular research requirements of the conjoined course, and a 25-50 page term paper in substitution of some portion of the conjoined course requirements, as determined by the instructor. Open to juniors and seniors only."
BU,Business,BU 0211,Legal Environment of Business,3 Credits,Junior standing.,None,None,None,"This course examines the broad philosophical as well as practical nature and function of the legal system, and introduces students to the legal and social responsibilities of business. The course includes an introduction to the legal system, the federal courts, Constitutional law, the United States Supreme Court, the civil process, and regulatory areas such as employment discrimination, protection of the environment, and corporate governance and securities markets."
BU,Business,BU 0220,Environmental Law and Policy,3 Credits,None,None,None,"EVME Environmental Studies Major Elective, EVPE Environmental Studies Elective, EVSS Environmental Studies: Social Science, MGEL Management: General Elective","This course surveys issues arising out of federal laws designed to protect the environment and manage resources. It considers in detail the role of the Environmental Protection Agency in the enforcement of environmental policies arising out of such laws as the National Environmental Policy Act, the Clean Water Act, and the Clear Air Act, among others. The course also considers the impact of Congress, political parties, bureaucracy, and interest groups in shaping environmental policy, giving special attention to the impact of environmental regulation on business and private property rights."
BU,Business,BU 0311,"The Law of Contracts, Sales, and Property",3 Credits,BU 0211.,None,None,None,"This course examines the components of common law contracts including the concepts of offer and acceptance, consideration, capacity and legality, assignment of rights and delegation of duties, as well as discharge of contracts. The course covers Articles 2 and 2A of the Uniform Commercial Code relating to leases, sales of goods, and warranties. The course also considers personal and real property, and bailments."
BU,Business,BU 0312,The Law of Business Organizations and Financial Transactions,3 Credits,BU 0211.,None,None,None,"This course offers an analysis of legal principles related to the law of agency, sole proprietorships, partnerships, corporations, limited liability companies, and other business forms. The second half of the course addresses several sections of the Uniform Commercial Code, such as negotiable instruments, bank collections and deposits and secured transactions. Finally, the course examines the law of suretyship, debtor-creditor relationships, and bankruptcy."
BU,Business,BU 0320,Employment Law and Discrimination in the Workplace,3 Credits,None,None,None,"MGEL Management: General Elective, UDIV U.S. Diversity","This course examines a variety of legal issues related to the workplace including the doctrine of employment at will, employee privacy, and the history and development of labor unions and the legal protections afforded by the National Labor Relations Act. A study of the

## Populate ERD Tables with data from Import Tables ##

In [43]:
%%sql

--Populating Course_Meeting table using import_course_meeting data-- 

INSERT INTO Course_Meeting (CRN, Term, Location, Start, End, Day)
SELECT DISTINCT crn, term, location, start, end, day
FROM import_course_meeting;

 * sqlite:///CourseData2.db
Done.


[]

In [44]:
%%sql 

--Checking data population of Course_Meeting table-- 

SELECT * 
FROM Course_Meeting
LIMIT 5;

 * sqlite:///CourseData2.db
Done.


MID,CRN,Term,Location,Start,End,DAY
1,70384,Fall2014,DSB 105,2014-09-02T08:00:00,2014-09-02T09:15:00,T
2,70384,Fall2014,DSB 105,2014-09-05T08:00:00,2014-09-05T09:15:00,F
3,70384,Fall2014,DSB 105,2014-09-09T08:00:00,2014-09-09T09:15:00,T
4,70384,Fall2014,DSB 105,2014-09-12T08:00:00,2014-09-12T09:15:00,F
5,70384,Fall2014,DSB 105,2014-09-16T08:00:00,2014-09-16T09:15:00,T


In [45]:
%%sql

--Populating PROGRAM table using import_course_catalog data-- 

INSERT INTO PROGRAM(PName, PCode)
SELECT DISTINCT program_name, program_code
FROM import_course_catalog;

 * sqlite:///CourseData2.db
Done.


[]

In [46]:
%%sql 

SELECT * 
FROM PROGRAM
LIMIT 10;

 * sqlite:///CourseData2.db
Done.


PID,PName,PCode
1,Asian Studies,AN
2,Business,BU
3,Black Studies,BL
4,Bioengineering,BEN
5,Arabic,AR
6,Anthropology,AY
7,American Studies,AS
8,Japanese,JA
9,Art History,AH
10,Applied Ethics,AE


In [47]:
%%sql 

--Populating Catalog table using import_course_catalog data--  

INSERT INTO CATALOG (CatID, Title, Description, Credits, PID, PREREQS, COREQS, FEES, Attribute)
SELECT DISTINCT catalog_id, course_title, description, credits, PID, prereqs, coreqs, fees, attributes
FROM import_course_catalog
    JOIN PROGRAM ON (import_course_catalog.program_name = program.PName);

 * sqlite:///CourseData2.db
Done.


[]

In [48]:
%%sql 

--Checking data population of Catalog table-- 

SELECT * 
FROM CATALOG
LIMIT 2;

 * sqlite:///CourseData2.db
Done.


CatClass,CatID,Title,Description,Credits,PID,PREREQS,COREQS,FEES,Attribute
1,AN 0301,Independent Study,Students undertake an individualized program of study in consultation with a director from the Asian studies faculty.,1-3 Credits,1,None,None,None,None
2,AN 0310,Asian Studies Seminar,"This seminar examines selected topics concerning Asia. This course is taught in conjunction with another 100-300 level course from a rotation of course offerings. Consult the Asian Studies director to identify the conjoined course for a given semester. The seminar concentrates on topics within the parameters of the conjoined course syllabus but adds research emphasis. Students registered for this course must complete a research project, to include 300-level research, in addition to the regular research requirements of the conjoined course, and a 25-50 page term paper in substitution of some portion of the conjoined course requirements, as determined by the instructor. Open to juniors and seniors only.",3 Credits,1,None,None,None,None


In [49]:
%%sql 

--Populating Instructor table using import_courses data--  

INSERT INTO Instructor (Name)
SELECT DISTINCT primary_instructor
FROM import_course
WHERE primary_instructor <> 'TBA' AND primary_instructor NOT LIKE 'primary_instructor' AND primary_instructor NOT LIKE '%/';

 * sqlite:///CourseData2.db
Done.


[]

In [50]:
%%sql 

--Checking data population of Instructor table-- 

SELECT * 
FROM Instructor
LIMIT 5;

 * sqlite:///CourseData2.db
Done.


InID,Name
1,Michael P. Coyne
2,Rebecca I. Bloch
3,Paul Caster
4,Jo Ann Drusbosky
5,Arleen N. Kardos


In [51]:
%%sql 

--Populating the SECTION table using import_course data--
--Joining Instructor table using import_course and instructor table--

INSERT INTO SECTION (CRN, Term, Section, Cap, Act, Rem, CatID, InID)
SELECT crn, term, section, cap, act, rem, catalog_id, InID
FROM import_course
    JOIN Instructor ON (import_course.primary_instructor = Instructor.Name)
    JOIN Catalog ON (import_course.catalog_id = Catalog.CatID)
;

 * sqlite:///CourseData2.db
Done.


[]

In [52]:
%%sql 

--Checking data population of Section table--

SELECT * 
FROM Section
LIMIT 30;

 * sqlite:///CourseData2.db
Done.


SID,CRN,Term,Section,Cap,Act,Rem,CatID,InID
1,70384,Fall2014,C01,0,31,-31,AC 0011,1
2,70385,Fall2014,C02,0,31,-31,AC 0011,1
3,70382,Fall2014,C03,0,31,-31,AC 0011,1
4,70291,Fall2014,C04,0,29,-29,AC 0011,2
5,70350,Fall2014,C05,0,30,-30,AC 0011,2
6,70381,Fall2014,C06,0,31,-31,AC 0011,2
7,70383,Fall2014,G,30,31,-1,AC 0011,3
8,70391,Fall2014,H,30,32,-2,AC 0011,4
9,71105,Fall2014,I,30,33,-3,AC 0011,4
10,71123,Fall2014,J,30,32,-2,AC 0011,4


## Drop Import Tables ##

In [53]:
%%sql
--Drop import tables--
DROP TABLE IF EXISTS import_course;
DROP TABLE IF EXISTS import_course_meeting;
DROP TABLE IF EXISTS import_course_catalog;

 * sqlite:///CourseData2.db
Done.
Done.
Done.


[]

In [54]:
%%sql
DROP TABLE IF EXISTS Instructor_dw;
DROP TABLE IF EXISTS COURSE;
DROP TABLE IF EXISTS MEETING;

 * sqlite:///CourseData2.db
Done.
Done.
Done.


[]

In [55]:
# Data Queries 

In [56]:
%%sql

/*What is Prof. Huntleys InID in our database?*/

SELECT * 
FROM INSTRUCTOR
WHERE NAME LIKE '%Huntley%';

 * sqlite:///CourseData2.db
Done.


InID,Name
274,Christopher L. Huntley


In [57]:
%%sql

--How many Catalog ID classes are 4 credits?-- 

SELECT COUNT(CatID) 
FROM CATALOG
WHERE Credits = '4 Credits' 
LIMIT 5;

 * sqlite:///CourseData2.db
Done.


COUNT(CatID)
55


In [58]:
%%sql

--How many insructors do we have in our database?-- 

SELECT COUNT(InID) 
FROM Instructor 
LIMIT 5;

 * sqlite:///CourseData2.db
Done.


COUNT(InID)
1103


In [59]:
%%sql 

--Which DSB classroom has the most class hours scheduled for the Spring 2019 semester?--

SELECT (SUM(strftime('%s',End)-strftime('%s',Start))/60)/60 AS CourseHours, CRN, Location, Term
FROM Course_Meeting
WHERE LOCATION LIKE '%DSB%' AND Term = 'Spring2019'
GROUP BY Location, Term
ORDER BY CourseHours DESC
LIMIT 10;

 * sqlite:///CourseData2.db
Done.


CourseHours,CRN,Location,Term
816,37510,DSB 105,Spring2019
644,37174,DSB 111,Spring2019
630,37512,DSB 106,Spring2019
602,37171,DSB 112,Spring2019
582,37364,DSB 109,Spring2019
545,37079,DSB 115,Spring2019
491,37295,DSB 104,Spring2019
473,37032,DSB 108,Spring2019
387,37257,DSB 110A,Spring2019
380,37366,DSB 110B,Spring2019


In [60]:
%%sql 

--Which professor has taught the most courses per Semester?-- 

SELECT InID, Term, Name, Count(Name) AS CoursesTaught, CatID
FROM Instructor
    JOIN SECTION USING (InID)
GROUP BY InID, Term
ORDER BY CoursesTaught DESC
LIMIT 10;

 * sqlite:///CourseData2.db
Done.


InID,Term,Name,CoursesTaught,CatID
173,Fall2017,Sonya R. Huber,32,ENW 0306
173,Spring2016,Sonya R. Huber,32,ENW 0206
173,Fall2015,Sonya R. Huber,31,ENW 0306
173,Fall2016,Sonya R. Huber,31,ENW 0346
173,Spring2017,Sonya R. Huber,31,ENW 0206
216,Spring2015,Michael C. White,29,ENW 0305
216,Fall2014,Michael C. White,27,ENW 0205
270,Fall2014,Laura Nash,21,HR 0202
635,Fall2016,Caitlin E. Hughes,18,MU 0060
270,Spring2015,Laura Nash,17,MU 0065


In [61]:
%%sql 

--What is the average class capacity where the capacity is greater than 10 students?--

SELECT AVG(CAP) as AverageCapacity
FROM SECTION 
WHERE CAP>10;

 * sqlite:///CourseData2.db
Done.


AverageCapacity
22.514390691977955


In [62]:
%%sql 

--Which day of the week overall has the most course hours taught--

SELECT ((SUM(strftime('%s',End)-strftime('%s',Start)))/60)/60 AS CourseHours, Day
FROM Course_Meeting
WHERE DAY <> 'day'
GROUP BY Day
ORDER BY CourseHours DESC;

 * sqlite:///CourseData2.db
Done.


CourseHours,DAY
116562,T
111245,R
108106,M
96924,W
68230,F
7143,S
956,U


In [63]:
# Huntley Queries!

In [64]:
%%sql 

--Which class has Prof. Huntley taught the most?-- 

SELECT CatID, Title, Name, COUNT(InID) AS TimesTaught
FROM Catalog
    JOIN SECTION USING (CatID)
    JOIN INSTRUCTOR USING (InID)
WHERE NAME LIKE "%Huntley%"
GROUP BY CatID
ORDER BY TimesTaught DESC
LIMIT 3;

 * sqlite:///CourseData2.db
Done.


CatID,Title,Name,TimesTaught
OM 0101,Operations Management,Christopher L. Huntley,7
IS 0240,Systems Analysis and Logical Design,Christopher L. Huntley,4
IS 0320,Systems Design and Implementation,Christopher L. Huntley,4


In [65]:
%%sql 

--How many times has Prof. Huntley taught IS 100-- 

SELECT TERM, Name, CatID
FROM SECTION 
    JOIN INSTRUCTOR USING (InID)
    JOIN CATALOG USING (CatID)
WHERE NAME LIKE '%Huntley'AND CatID = "IS 0100";

 * sqlite:///CourseData2.db
Done.


Term,Name,CatID
Fall2014,Christopher L. Huntley,IS 0100
Fall2015,Christopher L. Huntley,IS 0100
Fall2015,Christopher L. Huntley,IS 0100


In [66]:
%%sql 

--Which semester has Prof. Huntley taught the most courses?-- 

SELECT Name, COUNT(CatID) AS AmountOfClassesTaught, Term
FROM SECTION
    JOIN INSTRUCTOR USING (InID)
    JOIN Catalog USING (CatID)
WHERE NAME LIKE "%Huntley%"
GROUP BY Term
ORDER BY AmountOfClassesTaught DESC;

 * sqlite:///CourseData2.db
Done.


Name,AmountOfClassesTaught,Term
Christopher L. Huntley,4,Fall2015
Christopher L. Huntley,4,Fall2017
Christopher L. Huntley,4,Spring2017
Christopher L. Huntley,4,Spring2018
Christopher L. Huntley,3,Fall2014
Christopher L. Huntley,3,Spring2016
Christopher L. Huntley,2,Fall2016
Christopher L. Huntley,2,Fall2018
Christopher L. Huntley,2,Spring2019


In [67]:
%%sql 

--Which DSB room has Prof. Huntley taught the most in?--

SELECT Name, Location, COUNT(Location) AS NumClassMeetings
FROM Course_Meeting
    JOIN SECTION USING (CRN, Term)
    JOIN INSTRUCTOR USING (InID)
    JOIN Catalog USING (CatID)
WHERE NAME LIKE "%Huntley%"
GROUP BY Location
ORDER BY NumClassMeetings DESC;

 * sqlite:///CourseData2.db
Done.


Name,Location,NumClassMeetings
Christopher L. Huntley,DSB 106,108
Christopher L. Huntley,DSB 110B,105
Christopher L. Huntley,DSB 109,66
Christopher L. Huntley,DSB 115,59
Christopher L. Huntley,DSB 114,52
Christopher L. Huntley,DSB 108,29
Christopher L. Huntley,DSB 111,28
Christopher L. Huntley,DSB 112,27
Christopher L. Huntley,DSB 107,26
Christopher L. Huntley,DSB 110A,26


In [68]:
%%sql 

--Which day does Prof. Huntley historically teach the most?-- 

SELECT ((SUM(strftime('%s',End)-strftime('%s',Start)))/60)/60 AS CourseHours, Day, Name
FROM Course_Meeting
    JOIN SECTION USING (CRN, Term)
    JOIN Instructor USING (InID)
WHERE DAY <> 'day'AND Name LIKE '%Huntley%'
GROUP BY Day
ORDER BY CourseHours DESC;

 * sqlite:///CourseData2.db
Done.


CourseHours,DAY,Name
248,W,Christopher L. Huntley
234,T,Christopher L. Huntley
183,R,Christopher L. Huntley
173,M,Christopher L. Huntley
83,F,Christopher L. Huntley
48,S,Christopher L. Huntley
